In [1]:
!pip3 install bokeh

In [2]:
import numpy as np

In [3]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool

output_notebook()

Loading BokehJS ...

In [4]:
import pandas as pd
fields = ['WEIGHT', 'ABOID', 'AGEGRP', 'ATTSCH', 'BedRm', 'BFNMEMB', 'CFInc',
       'CFInc_AT', 'CfSize', 'CFSTAT', 'CIP2011', 'CIP2011_STEM_SUM',
       'Citizen', 'CitOth', 'CMA', 'CONDO', 'COW', 'DETH123', 'DPGRSUM',
       'DTYPE', 'EFDecile', 'EfDIMBM', 'EFInc', 'EFInc_AT', 'EfSize',
       'ETHDER', 'GENSTAT', 'HCORENEED_IND', 'HDGREE', 'HHInc',
       'HHInc_AT', 'HHMRKINC', 'HHSIZE', 'HHTYPE',
       'HLANO', 'IMMCAT5', 'IMMSTAT',
       'LFACT', 'LICO', 'LICO_AT', 'LOC_ST_RES', 'LoLIMA', 'LoLIMB',
       'LoMBM', 'LSTWRK', 'MarStH', 'MOB1', 'Mob5',
       'MTNNO', 'NOS', 'PKID0_1', 'PKID15_24', 'PKID2_5',
       'PKID25', 'PKID6_14', 'PKIDS', 'POB', 'POBF', 'POBM', 'PR', 'PR1',
       'PR5', 'PRIHM', 'REGIND', 'REPAIR', 'ROOMS', 'Sex', 'SHELCO',
       'SSGRAD', 'Tenur', 'VisMin', 'WRKACT']

dataset = pd.read_csv("pumf-2016-selected-features-No-MTEn-English.csv", usecols=fields)
dataset.head()
# removed market income (MrkInc, TotInc, TotInAT)

,WEIGHT,ABOID,AGEGRP,ATTSCH,BedRm,BFNMEMB,CFInc,CFInc_AT,CfSize,CFSTAT,...,PRIHM,REGIND,REPAIR,ROOMS,Sex,SHELCO,SSGRAD,Tenur,VisMin,WRKACT
0,37.120914,6,15,1,2,0,16,15,2,1,...,1,0,3,5,1,700,6,1,13,9
1,37.019784,6,18,1,3,0,13,12,1,6,...,1,0,1,5,1,900,6,1,13,1
2,37.041550,6,14,1,2,0,25,21,2,1,...,0,0,1,4,2,900,5,2,13,11
3,37.042280,6,11,1,3,0,28,25,4,2,...,1,0,1,8,2,2200,8,1,12,11
4,37.037277,6,12,1,4,0,29,25,4,2,...,0,0,1,10,1,2700,8,1,2,1


In [5]:
motherTongueDict = {}
motherTongueDict['1'] = 'No non-official languages'
motherTongueDict['2'] = 'Aboriginal languages'
motherTongueDict['3'] = 'Arabic'
motherTongueDict['4'] = 'Mandarin'
motherTongueDict['5'] = 'Cantonese'
motherTongueDict['6'] = 'Chinese languages'
motherTongueDict['7'] = 'German'
motherTongueDict['8'] = 'Other European languages' #'Other Germanic languages'
motherTongueDict['9'] = 'Other European languages' #'Greek'
motherTongueDict['10'] = 'Urdu'
motherTongueDict['11'] = 'Persian (Farsi)'
motherTongueDict['12'] = 'Other Indo-Iranian languages'
motherTongueDict['13'] = 'Italian'
motherTongueDict['14'] = 'Polish'
motherTongueDict['15'] = 'Portuguese'
motherTongueDict['16'] = 'Punjabi (Panjabi)'
motherTongueDict['17'] = 'Spanish'
motherTongueDict['18'] = 'Slavic languages' #'Ukrainian'
motherTongueDict['19'] = 'Austro-Asiatic languages' #'Vietnamese'
motherTongueDict['20'] = 'Austro-Asiatic languages'
motherTongueDict['21'] = 'Other European languages'
motherTongueDict['22'] = 'Russian'
motherTongueDict['23'] = 'Slavic languages'
motherTongueDict['24'] = 'Other European languages' #'Uralic languages'
motherTongueDict['25'] = 'Other Afro-Asiatic and African languages' #'Other Afro-Asiatic languages'
motherTongueDict['26'] = 'Tamil'
motherTongueDict['27'] = 'All other languages' #'Other Dravidian languages'
motherTongueDict['28'] = 'Other East and Southeast Asian languages' #'Korean'
motherTongueDict['29'] = 'Other East and Southeast Asian languages'
motherTongueDict['30'] = 'Tagalog (Pilipino, Filipino)'
motherTongueDict['31'] = 'Other Afro-Asiatic and African languages' #'Niger-Congo languages and other African languages'
motherTongueDict['32'] = 'All other languages' #'All other single languages'
motherTongueDict['88'] = 'Not available'

In [6]:
homeLanguageDict = {}
homeLanguageDict['1'] = 'No non-official languages'
homeLanguageDict['2'] = 'Aboriginal languages'
homeLanguageDict['3'] = 'Italian'
homeLanguageDict['4'] = 'Spanish'
homeLanguageDict['5'] = 'Portugese'
homeLanguageDict['6'] = 'German'
homeLanguageDict['7'] = 'Russian'
homeLanguageDict['8'] = 'Polish'
homeLanguageDict['9'] = 'Slavic languages'
homeLanguageDict['10'] = 'Other European languages'
homeLanguageDict['11'] = 'Arabic'
homeLanguageDict['12'] = 'Other Afro-Asiatic and African languages'
homeLanguageDict['13'] = 'Punjabi (Panjabi)'
homeLanguageDict['14'] = 'Urdu'
homeLanguageDict['15'] = 'Persian (Farsi)'
homeLanguageDict['16'] = 'Other Indo-Iranian languages'
homeLanguageDict['17'] = 'Cantonese'
homeLanguageDict['18'] = 'Mandarin'
homeLanguageDict['19'] = 'Chinese languages'
homeLanguageDict['20'] = 'Austro-Asiatic languages'
homeLanguageDict['21'] = 'Tagalog (Pilipino, Filipino)'
homeLanguageDict['22'] = 'Other East and Southeast Asian languages'
homeLanguageDict['23'] = 'Tamil'
homeLanguageDict['24'] = 'All other languages'
homeLanguageDict['88'] = 'Not available'

In [7]:
dataset = dataset[dataset.HLANO != 88]
dataset = dataset[dataset.MTNNO != 88]
# class label: home language part A - first language write in component
homeLang = dataset['HLANO']
# class label: mother tongue part A - first language write in component
motherTongue = dataset['MTNNO']
fields.remove('HLANO')
fields.remove('MTNNO')
homeLanguageMapped = homeLang.apply(lambda x: homeLanguageDict[str(x)])
motherTongueMapped = motherTongue.apply(lambda x: motherTongueDict[str(x)])
languageShift = homeLanguageMapped.ne(motherTongueMapped)

fields.remove('WEIGHT')
x = dataset[fields]
weights = dataset["WEIGHT"]

In [8]:
homeLanguageMapped

0         No non-official languages
1         No non-official languages
2         No non-official languages
3         No non-official languages
4         No non-official languages
                    ...            
387812    No non-official languages
387813    No non-official languages
387814    No non-official languages
387815    No non-official languages
387816                    Cantonese
Name: HLANO, Length: 379075, dtype: object

In [9]:
motherTongueMapped

0            No non-official languages
1            No non-official languages
2            No non-official languages
3                            Cantonese
4                             Mandarin
                      ...             
387812    Tagalog (Pilipino, Filipino)
387813       No non-official languages
387814       No non-official languages
387815       No non-official languages
387816                       Cantonese
Name: MTNNO, Length: 379075, dtype: object

In [10]:
languageShift

0         False
1         False
2         False
3          True
4          True
          ...  
387812     True
387813    False
387814    False
387815    False
387816    False
Length: 379075, dtype: bool

In [11]:
x

,ABOID,AGEGRP,ATTSCH,BedRm,BFNMEMB,CFInc,CFInc_AT,CfSize,CFSTAT,CIP2011,...,PRIHM,REGIND,REPAIR,ROOMS,Sex,SHELCO,SSGRAD,Tenur,VisMin,WRKACT
0,6,15,1,2,0,16,15,2,1,5,...,1,0,3,5,1,700,6,1,13,9
1,6,18,1,3,0,13,12,1,6,5,...,1,0,1,5,1,900,6,1,13,1
2,6,14,1,2,0,25,21,2,1,11,...,0,0,1,4,2,900,5,2,13,11
3,6,11,1,3,0,28,25,4,2,7,...,1,0,1,8,2,2200,8,1,12,11
4,6,12,1,4,0,29,25,4,2,6,...,0,0,1,10,1,2700,8,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387812,6,14,1,3,0,25,21,3,2,7,...,1,0,1,6,2,1500,8,1,4,11
387813,6,11,1,4,0,30,25,4,2,4,...,1,0,1,7,1,1200,5,1,13,10
387814,6,11,1,4,0,30,25,4,2,8,...,0,0,1,7,2,1400,5,1,13,5
387815,6,3,9,4,0,30,25,4,4,99,...,0,0,1,7,2,1300,99,1,13,99


In [12]:
# discretization

def discretify(dataset, quantiles=10, target_fields=('SHELCO', 'MrkInc', 'TotInc', 'TotInc_AT', 'VALUE'),
               invalid_quantile = 88):
    """Convert target fields in dataset to quantile categories."""
    
    invalid_values = {
        'SHELCO': (),
        'MrkInc': (88888888, 99999999),
        'TotInc': (88888888, 99999999),
        'TotInc_AT': (88888888, 99999999),
        'VALUE': (88888888, 99999999)
    }
    
    dataset_copy = dataset.copy()

    for continuous in target_fields:
        if continuous in dataset_copy:
            print(continuous)
            values = []
            quantile_values = pd.qcut(dataset_copy[continuous].rank(method='first'), quantiles, False)

            for value, quantile in zip(dataset_copy[continuous], quantile_values):
                if value in invalid_values[continuous]:
                    values.append(invalid_quantile)
                else:
                    values.append(quantile)

            dataset_copy[continuous] = values
    
    return dataset_copy


x = discretify(x)

x

SHELCO


,ABOID,AGEGRP,ATTSCH,BedRm,BFNMEMB,CFInc,CFInc_AT,CfSize,CFSTAT,CIP2011,...,PRIHM,REGIND,REPAIR,ROOMS,Sex,SHELCO,SSGRAD,Tenur,VisMin,WRKACT
0,6,15,1,2,0,16,15,2,1,5,...,1,0,3,5,1,1,6,1,13,9
1,6,18,1,3,0,13,12,1,6,5,...,1,0,1,5,1,3,6,1,13,1
2,6,14,1,2,0,25,21,2,1,11,...,0,0,1,4,2,3,5,2,13,11
3,6,11,1,3,0,28,25,4,2,7,...,1,0,1,8,2,8,8,1,12,11
4,6,12,1,4,0,29,25,4,2,6,...,0,0,1,10,1,9,8,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387812,6,14,1,3,0,25,21,3,2,7,...,1,0,1,6,2,6,8,1,4,11
387813,6,11,1,4,0,30,25,4,2,4,...,1,0,1,7,1,5,5,1,13,10
387814,6,11,1,4,0,30,25,4,2,8,...,0,0,1,7,2,6,5,1,13,5
387815,6,3,9,4,0,30,25,4,4,99,...,0,0,1,7,2,5,99,1,13,99


In [13]:
weights

0         37.120914
1         37.019784
2         37.041550
3         37.042280
4         37.037277
            ...    
387812    37.041550
387813    37.041550
387814    37.041550
387815    37.041550
387816    37.037277
Name: WEIGHT, Length: 379075, dtype: float64

In [14]:
!pip3 install sklearn

In [15]:
import sys 
sys.version

'3.7.0 (v3.7.0:1bf9cc5093, Jun 27 2018, 04:59:51) [MSC v.1914 64 bit (AMD64)]'

In [16]:
from sklearn import datasets

In [17]:
from sklearn.naive_bayes import CategoricalNB
classifier = CategoricalNB()

In [18]:
from sklearn.model_selection import train_test_split
weights_train, weights_test, x_train, x_test, languageShift_train, languageShift_test = train_test_split(weights, x, languageShift)

In [19]:
x_train[(x_train < 0).any(1)]

,ABOID,AGEGRP,ATTSCH,BedRm,BFNMEMB,CFInc,CFInc_AT,CfSize,CFSTAT,CIP2011,...,PRIHM,REGIND,REPAIR,ROOMS,Sex,SHELCO,SSGRAD,Tenur,VisMin,WRKACT


In [20]:
classifier.fit(x_train, languageShift_train, sample_weight=weights_train.tolist())

CategoricalNB(alpha=1.0, class_prior=None, fit_prior=True)

In [21]:
languageShift_pred = classifier.predict(x_test)

languageShift_probs = classifier.predict_proba(x_test)[:, 1]

In [22]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

The Confusion Matrix indicates, given the predicted language (row names), for how many records did the algorith match the actual language. The classification report and accuracy score are other ways to evaluate how well the classification algorithm performed.

In [23]:
confusionMatrixLanguageShift = pd.DataFrame(
    confusion_matrix(languageShift_test,languageShift_pred), 
    index=['False', 'True'], 
    columns=['False', 'True'], 
)

In [24]:
print(confusionMatrixLanguageShift)
print(classification_report(languageShift_test,languageShift_pred,target_names=['False', 'True']))
print(accuracy_score(languageShift_test, languageShift_pred))

       False   True
False  52400  24464
True    3598  14307
              precision    recall  f1-score   support

       False       0.94      0.68      0.79     76864
        True       0.37      0.80      0.50     17905

    accuracy                           0.70     94769
   macro avg       0.65      0.74      0.65     94769
weighted avg       0.83      0.70      0.74     94769

0.7038905127204044


In [25]:
from sklearn.metrics import roc_auc_score, roc_curve

auc_score = roc_auc_score(languageShift_test, languageShift_probs)
auc_score

0.8190072061911545

In [26]:
base_fpr, base_tpr, _ = roc_curve(languageShift_test, [1 for _ in range(len(languageShift_test))])
model_fpr, model_tpr, _ = roc_curve(languageShift_test, languageShift_probs)

# Generate a figure container
plot = figure(plot_height=400,
              plot_width=400,
              title='ROC AUC Score',
              x_axis_label='False Positive Rate',
              y_axis_label='True Positive Rate',
              x_range=[0, 1],
              y_range=[0, 1])

plot.line(model_fpr, model_tpr, line_width=3, color='red', legend_label='Naive Bayes')
plot.line(base_fpr, base_tpr, line_width=3, line_dash='dashed', legend_label='Random')

plot.title.align = 'center'
plot.legend.location = 'bottom_right'

show(plot)

In [27]:
!pip install tabulate

In [28]:
classifier.classes_

array([False,  True])

In [29]:
from tabulate import tabulate
importances = classifier.feature_log_prob_
print(importances)
#headers = ["name", "prob of feature given class: False", "prob of feature given class: True"]
#values = sorted(zip(x_train.columns, np.exp(importances[0]), np.exp(importances[1])), key=lambda x: (x[2] * -1, x[1] * -1))
#print(tabulate(values, headers, tablefmt="plain"))

[array([[-15.95686571,  -4.40897498,  -4.64595532,  -5.95791558,
         -8.02701995,  -7.80124567,  -0.02540925],
       [-14.51831477,  -3.7708063 ,  -6.58869956,  -6.00084152,
         -9.11016489,  -8.82356383,  -0.02751944]]), array([[-15.95687535,  -2.95073096,  -3.88298691,  -3.45007361,
         -3.93906751,  -3.56825339,  -3.53532099,  -3.84891456,
         -2.83845236,  -2.83267723,  -2.73431671,  -2.68584798,
         -2.69521345,  -2.66956799,  -2.51704016,  -2.54176513,
         -2.68972224,  -2.81731952,  -3.11886002,  -3.42342262,
         -3.84361553,  -4.13966879, -15.95687535, -15.95687535,
        -15.95687535, -15.95687535, -15.95687535, -15.95687535,
        -15.95687535, -15.95687535, -15.95687535, -15.95687535,
        -15.95687535, -15.95687535, -15.95687535, -15.95687535,
        -15.95687535, -15.95687535, -15.95687535, -15.95687535,
        -15.95687535, -15.95687535, -15.95687535, -15.95687535,
        -15.95687535, -15.95687535, -15.95687535, -15.95687535,

In [30]:
!pip install bokeh

In [31]:
#from sklearn.tree import export_graphviz
#estimator = classifier.estimators_[5]
# Creates dot file named randomForestTreeSample.dot
#export_graphviz(estimator, 
#                out_file='randomForestTreeSample.dot', 
#                feature_names = list(x.columns),
#                class_names = ['False', 'True'],
#                rounded = True, proportion = False, 
#                precision = 2, filled = True,
#                max_depth=5)